In [5]:
import functools
import itertools
import math
import re
import statistics
from functools import reduce
from pprint import pprint
from typing import (
    List,
    Optional,
    Generator,
    NamedTuple,
    Tuple,
    Set,
    Any,
    Dict,
    Union,
    Callable,
    Iterable,
    Sequence,
)

import rich.panel
import sklearn.base
from alive_progress import alive_bar
from numpy.distutils.misc_util import is_sequence
from tqdm import tqdm, trange
from log_symbols import LogSymbols

import overrides as overrides
import pandas as pds
import tabulate as tabulate
from collections import namedtuple
from sklearn import linear_model, metrics, preprocessing, model_selection, svm
from tabulate import tabulate
from pathlib import Path
import pickle
from halo import Halo
import random
import argparse
import sys
from sklearn.experimental import enable_halving_search_cv
from sklearn import neural_network, tree
from sklearn.model_selection import (
    HalvingGridSearchCV,
    RandomizedSearchCV,
    HalvingRandomSearchCV,
)
import scipy.stats as scistats

import rich.progress
import commons
from data_types import PickleOut
from rich.traceback import install as niceTracebacks
from rich.table import Table as rTable

from db_actions import db_actions
import numpy as np

from linRegPred import getSplits, getTransformers
from scipy.stats import rankdata
from sklearn.metrics import r2_score
import textdistance as TD

In [6]:
with db_actions.connect() as conn:
    dF = pds.read_sql('SELECT * FROM "runtimeScorePredBase1000"', conn)
# print(dF)
all_x_cols = [
    "build-linux-kernel1",
    "fio2",
    "fio3",
    "fio4",
    "fio5",
    "fio6",
    "fio7",
    "fio8",
    "fio9",
    "iperf10",
    "iperf11",
    "iperf12",
    "iperf13",
    "john-the-ripper14",
    "john-the-ripper15",
    "ramspeed16",
    "ramspeed17",
    "ramspeed18",
    "ramspeed19",
    "ramspeed20",
    "ramspeed21",
    "ramspeed22",
    "ramspeed23",
    "ramspeed24",
    "ramspeed25",
    "stream26",
    "stream27",
    "stream28",
    "stream29",
    "pCpu",
    "cpus",
    "rss",
    "vmem",
    "rchar",
    "wchar",
    "syscr",
    "syscw",
]
filtered_x_cols = ['john-the-ripper15', 'ramspeed17', 'ramspeed18', 'ramspeed19', 'ramspeed20', 'ramspeed21',
                   'ramspeed23', 'ramspeed24', 'ramspeed25', 'pCpu', 'cpus', 'vmem', 'rchar', 'wchar', 'syscr',
                   'syscw']
x_cols = all_x_cols
y_cols = "realtime"

X: pds.DataFrame = dF[x_cols]
y: pds.DataFrame = dF[y_cols]

In [7]:
from my_yaml import yaml_load

with open("nodeConfigIdLookup.yaml", "r") as f:
    t = yaml_load(f)
    nodeIDLUT = {v: k for k, v in t.items()}
dF["nodeName"] = dF["nodeConfig"].transform(lambda x: nodeIDLUT[x])

In [8]:
awsGroupLUT = {
    "general": ["M7g", "Mac", "M6g", "M6i", "M6in", "M6a", "M5", "M5n", "M5zn", "M5a", "M4", "A1", "T4g", "T3", "T3a",
                "T2"],
    "compute": ["C7g", "C7gn", "C6i", "C6in", "C6a", "C6g", "C6gn", "C5", "C5n", "C5a", "C4"],
    "memory": ["R7g", "R7iz", "R6g", "R6i", "R6in", "R6a", "R5", "R5n", "R5b", "R5a", "R4", "X2gd", "X2idn", "X2iedn",
               "X2iezn", "X1", "X1e", "z1d"],
    "accelerated": ["P4", "P3", "P2", "DL1", "Trn1", "Inf2", "Inf1", "G5", "G5g", "G4dn", "G4ad", "G3", "F1", "VT1"],
    "storage": ["Im4gn", "Is4gen", "I4i", "I3", "I3en", "D2", "D3", "D3en", "H1"],
    "HPC": ["Hpc6id", "Hpc6a"]
}

In [9]:
awsInstanceGroupLUT = {i.lower(): k for k, v in awsGroupLUT.items() for i in v}

In [10]:
dF["nodeGroup"] = dF["nodeName"].transform(lambda x: awsInstanceGroupLUT[x.split(".")[0]])

In [11]:
usedAwsGroupLUT = {
    k: list(pds.unique(dF.query("nodeGroup==@k").nodeName.values)) for k in awsGroupLUT.keys()
}

In [12]:
nodesPerGroup = {
    k: len(v) for k, v in usedAwsGroupLUT.items()
}

In [13]:
import seaborn as sns

if False:
    #sns.boxplot(data=dF,y="realtime",x="taskName",hue="nodeGroup")
    for wf in dF.groupby(by="wfName"):
        wfname, data = wf
        print(wfname)
        grid = sns.FacetGrid(data, row="wfName", col="nodeGroup", sharex=False, sharey=True, margin_titles=True,
                             height=5)
        grid.map_dataframe(sns.boxplot, y="realtime", x="taskName")

# Find prototypes for each group

In [83]:
true_ranks_df = pds.DataFrame()
for l, d in dF.groupby(by=["wfName", "taskName", "nodeName"]):
    wn, tn, nn = l
    true_ranks_df = true_ranks_df.append(
        {"wfName": wn, "taskName": tn, "nodeName": nn, "realtime": d.realtime.mean(), "rank": 0},
        ignore_index=True)

for l, d in true_ranks_df.groupby(by=["wfName", "taskName"]):
    wn, tn = l
    ranks = rankdata(d.realtime.values, method="min")
    d["rank"] = ranks
    true_ranks_df.update(d)

In [85]:
with db_actions.connect() as conn:
    scores_df = pds.read_sql('SELECT * FROM "nodeBenchmarkTransposedScores"', conn)


In [86]:
scores_df["nodeName"] = scores_df["nodeConfig"].transform(lambda x: nodeIDLUT[x])
scores_df["nodeGroup"] = scores_df["nodeName"].transform(lambda x: awsInstanceGroupLUT[x.split(".")[0]])

In [17]:
scores = dict()
benchmarkCols = [
    "build-linux-kernel1",
    "fio2",
    "fio3",
    "fio4",
    "fio5",
    "fio6",
    "fio7",
    "fio8",
    "fio9",
    "iperf10",
    "iperf11",
    "iperf12",
    "iperf13",
    "john-the-ripper14",
    "john-the-ripper15",
    "ramspeed16",
    "ramspeed17",
    "ramspeed18",
    "ramspeed19",
    "ramspeed20",
    "ramspeed21",
    "ramspeed22",
    "ramspeed23",
    "ramspeed24",
    "ramspeed25",
    "stream26",
    "stream27",
    "stream28",
    "stream29"]
nodeGroups = [scores_df.query("nodeGroup==@gn").nodeName.values for gn in groupOrder]
nodeGroupBenchmarkAverages = [np.average([scores_df.query("nodeName==@nn")[benchmarkCols] for nn in ng], axis=0) for ng
                              in nodeGroups]
for c in itertools.product(*nodeGroups):
    n1, n2, n3, n4 = c
    ns1 = scores_df.query("nodeName == @n1")[benchmarkCols].values[0]
    ns2 = scores_df.query("nodeName == @n2")[benchmarkCols].values[0]
    ns3 = scores_df.query("nodeName == @n3")[benchmarkCols].values[0]
    ns4 = scores_df.query("nodeName == @n4")[benchmarkCols].values[0]
    distance_between_protos = np.linalg.norm(ns1 - ns2) + np.linalg.norm(ns1 - ns3) + np.linalg.norm(
        ns1 - ns4) + np.linalg.norm(ns2 - ns3) + np.linalg.norm(ns2 - ns4) + np.linalg.norm(ns3 - ns4)
    distances_from_group_averages = [np.linalg.norm(bs - ngba) for bs, ngba in
                                     zip([ns1, ns2, ns3, ns4], nodeGroupBenchmarkAverages)]
    scores[c] = (distance_between_protos, np.sum(distances_from_group_averages))

In [ ]:
factor = sum(x[0] for x in scores.values()) / sum(x[1] for x in scores.values())
scores = {
    k: v[0] + factor * v[1] for k, v in scores.items()
}

In [ ]:
cs, ds = scores.keys(), scores.values()
#print("\n".join(map(lambda x: repr(x), sorted(zip(cs, ds), key=lambda x: x[1])[:10])))
groupPrototypes = sorted(zip(cs, ds), key=lambda x: x[1])[0][0]
print(groupPrototypes)

In [ ]:
bestGroupPrototypes = (
    'm5.xlarge', 'c5.xlarge', 'r5.xlarge', 'i3.2xlarge')  # = sorted(zip(cs, ds), key=lambda x: x[1])[0][0]
bestGroupPrototypesLUT = {
    awsInstanceGroupLUT[k.split(".")[0]]: k for k in bestGroupPrototypes
}

In [21]:
print(bestGroupPrototypesLUT)

{'general': 'm5.xlarge', 'compute': 'c5a.2xlarge', 'memory': 'r5.xlarge', 'storage': 'i3.2xlarge'}


# Produce Group Rankings

In [76]:
groupOrder = ["general", "compute", "memory", "storage"]
nodeOrder = list(pds.unique(dF.nodeName))

In [33]:
def makeGroupRankings(prototypes):
    prototypesLUT = {
        awsInstanceGroupLUT[k.split(".")[0]]: k for k in prototypes
    }
    groupPerformances = dict()
    for g in dF.groupby(by=["nodeGroup", "wfName", "taskName"]):
        l, d = g
        ng, wfn, tn = l
        if (wfn, tn) not in groupPerformances.keys():
            groupPerformances[(wfn, tn)] = dict()
        groupProto = prototypesLUT[ng]
        groupPerformances[(wfn, tn)][ng] = d.query("nodeName == @groupProto").realtime.mean()
        #groupPerformances[l] = d.realtime.mean()
    groupRankings = pds.DataFrame(columns=["wfName", "taskName", *groupOrder])
    for k, v in groupPerformances.items():
        wfn, tn = k
        performances = [v[i] for i in groupOrder]
        ranks = rankdata(performances, method="min")
        groupRankings = groupRankings.append(
            {"wfName": wfn, "taskName": tn, **{o: r for o, r in zip(groupOrder, ranks)}},
            ignore_index=True)
    return groupRankings


#### Produce intragroup node rankings

In [69]:
def makeNodeRankingsInGroups(groupRankings):
    tmp = pds.DataFrame(
        columns=["group", "nodeName", "wfName", "taskName", "realtime", "groupRank", "intraGroupRank", "globalRank",
                 "trueRank"])
    for g in dF.groupby(by=["nodeGroup", "wfName", "taskName", "nodeName"]):
        l, d = g
        ng, wfn, tn, nn = l
        tmp = tmp.append({"group": ng, "nodeName": nn, "wfName": wfn, "taskName": tn, "realtime": d.realtime.mean()},
                         ignore_index=True)

    for l, d in tmp.groupby(["group", "wfName", "taskName"]):
        gr, wfn, tn = l
        ra = list(rankdata(d.realtime, method="min"))
        cache = groupRankings.query("wfName==@wfn and taskName==@tn")
        ar = 0
        grr = cache[gr].values[0]
        for k, v in sorted(list(zip(groupOrder, (cache[k].values[0] for k in groupOrder))), key=lambda x: x[1]):
            if v >= grr:
                break
            ar += nodesPerGroup[k]
        d["intraGroupRank"] = ra
        d["groupRank"] = [grr] * len(ra)
        d["globalRank"] = np.array([ar] * len(ra)) + np.array(ra)
        tmp.update(d)

    for l, d in tmp.groupby(["wfName", "taskName"]):
        tr = list(rankdata(d.realtime, method="min"))
        d["trueRank"] = tr
        tmp.update(d)

    return tmp

In [77]:
tmp3 = makeNodeRankingsInGroups(makeGroupRankings(bestGroupPrototypes))

In [78]:
export = pds.DataFrame(
    {"wfName": tmp3.wfName, "taskName": tmp3.taskName, "nodeName": tmp3.nodeName, "rank": tmp3.globalRank})

In [79]:
export.to_csv("no_ML_ranks.csv", index=False)

In [80]:
scores = []
for l, d in tmp3.groupby(by=["wfName", "taskName"]):
    scores.append(commons.rankingScore(d.globalRank.values, d.trueRank.values))

In [88]:
def scoreRankings(ranking_df):
    scores = []
    for l, d in ranking_df.groupby(by=["wfName", "taskName"]):
        scores.append(commons.rankingScore(d.globalRank.values, d.trueRank.values))
    return np.average(scores), np.std(scores)

In [89]:
print(scoreRankings(tmp3))

(0.7777256863051519, 0.04053200853682953)


# Bruteforce test all prototype selections

In [92]:
nodeGroups = [scores_df.query("nodeGroup==@gn").nodeName.values for gn in groupOrder]
prototypeSelectionScores = dict()
o = 0
total = reduce(lambda a, x: a * len(x), nodeGroups, 1)
for c in itertools.product(*nodeGroups):
    prototypeSelectionScores[c] = scoreRankings(makeNodeRankingsInGroups(makeGroupRankings(c)))
    o += 1
    print(f"{o}/{total}")

1/1620
2/1620
3/1620
4/1620
5/1620
6/1620
7/1620
8/1620
9/1620
10/1620
11/1620
12/1620
13/1620
14/1620
15/1620
16/1620
17/1620
18/1620
19/1620
20/1620
21/1620
22/1620
23/1620
24/1620
25/1620
26/1620
27/1620
28/1620
29/1620
30/1620
31/1620
32/1620
33/1620
34/1620
35/1620
36/1620
37/1620
38/1620
39/1620
40/1620
41/1620
42/1620
43/1620
44/1620
45/1620
46/1620
47/1620
48/1620
49/1620
50/1620
51/1620
52/1620
53/1620
54/1620
55/1620
56/1620
57/1620
58/1620
59/1620
60/1620
61/1620
62/1620
63/1620
64/1620
65/1620
66/1620
67/1620
68/1620
69/1620
70/1620
71/1620
72/1620
73/1620
74/1620
75/1620
76/1620
77/1620
78/1620
79/1620
80/1620
81/1620
82/1620
83/1620
84/1620
85/1620
86/1620
87/1620
88/1620
89/1620
90/1620
91/1620
92/1620
93/1620
94/1620
95/1620
96/1620
97/1620
98/1620
99/1620
100/1620


╭──────────────────────────── Traceback (most recent call last) ────────────────────────────╮
│ /tmp/ipykernel_13909/3623794660.py:6 in <module>                                          │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_13909/3623794660.py'                 │
│                                                                                           │
│ /tmp/ipykernel_13909/2585536469.py:22 in makeGroupRankings                                │
│                                                                                           │
│ [Errno 2] No such file or directory: '/tmp/ipykernel_13909/2585536469.py'                 │
│                                                                                           │
│ /home/alex/anaconda3/envs/py/lib/python3.9/site-packages/pandas/core/frame.py:8961 in     │
│ append                                                                                    │
│                                                                                           │
│    8958 │   │   else:                                                                     │
│    8959 │   │   │   to_concat = [self, other]                                             │
│    8960 │   │   return (                                                                  │
│ ❱  8961 │   │   │   concat(                                                               │
│    8962 │   │   │   │   to_concat,                                                        │
│    8963 │   │   │   │   ignore_index=ignore_index,                                        │
│    8964 │   │   │   │   verify_integrity=verify_integrity,                                │
│                                                                                           │
│ /home/alex/anaconda3/envs/py/lib/python3.9/site-packages/pandas/util/_decorators.py:311   │
│ in wrapper                                                                                │
│                                                                                           │
│   308 │   │   │   │   │   FutureWarning,                                                  │
│   309 │   │   │   │   │   stacklevel=stacklevel,                                          │
│   310 │   │   │   │   )                                                                   │
│ ❱ 311 │   │   │   return func(*args, **kwargs)                                            │
│   312 │   │                                                                               │
│   313 │   │   return wrapper                                                              │
│   314                                                                                     │
│                                                                                           │
│ /home/alex/anaconda3/envs/py/lib/python3.9/site-packages/pandas/core/reshape/concat.py:29 │
│ 4 in concat                                                                               │
│                                                                                           │
│   291 │   │   ...                                                                         │
│   292 │   ValueError: Indexes have overlapping values: ['a']                              │
│   293 │   """                                                                             │
│ ❱ 294 │   op = _Concatenator(                                                             │
│   295 │   │   objs,                                                                       │
│   296 │   │   axis=axis,                                                                  │
│   297 │   │   ignore_index=ignore_index,                                                  │
│                                                                                           │
│ /home/alex/anaconda3/envs/py/lib/python3.9/site-pa

In [97]:
print(sorted(zip(prototypeSelectionScores.keys(), prototypeSelectionScores.values()),key=lambda x:prototypeSelectionScores[x[0]][0],reverse=True))

[(('m5.large', 'c5.2xlarge', 'r5.large', 'i3.large'), (0.7795315240228508, 0.045679419955128885)), (('m5.large', 'c5.2xlarge', 'r5.large', 'i3.xlarge'), (0.7790106092965915, 0.04536687791506502)), (('m5.large', 'c5a.xlarge', 'r5.large', 'i3.large'), (0.7771526801062666, 0.0498081289858349)), (('m5.large', 'c5.xlarge', 'r5.large', 'i3.large'), (0.7767706759736764, 0.046822066322327716)), (('m5.large', 'c5a.xlarge', 'r5.large', 'i3.xlarge'), (0.7766317653800072, 0.04949662178106412)), (('m5.large', 'c5.xlarge', 'r5.large', 'i3.xlarge'), (0.7762497612474171, 0.04648627503042468)), (('m5.large', 'c5.2xlarge', 'r5.xlarge', 'i3.xlarge'), (0.7760934868295394, 0.03996418338376854)), (('m5.large', 'c5a.xlarge', 'r5.xlarge', 'i3.xlarge'), (0.775867757114827, 0.04146157636297061)), (('m5.large', 'c5.2xlarge', 'r5.large', 'i3.2xlarge'), (0.7757114826969491, 0.04858666717223046)), (('m5.large', 'c5.xlarge', 'r5.xlarge', 'i3.xlarge'), (0.7744786511781355, 0.041069133441499435)), (('m5.large', 'c5a.x